In [17]:
import pandas as pd

In [110]:
pcode = pd.read_csv("https://raw.githubusercontent.com/atqnp/postcode-malaysia/master/postcode/postcode_my.csv")
pcode.drop(pcode.columns[0], axis=1, inplace=True)
pcode.head()

,Location,Post_Office,Postcode,State
0,Kangar - Peti surat,Kangar,01007,Perlis
1,Kangar - Beg berkunci,Kangar,01009,Perlis
2,Alor Redeh,Kangar,01000,Perlis
3,Alor Tok Terang,Kangar,01000,Perlis
4,Bohor Gelam,Kangar,01000,Perlis


In [111]:
pcode['locality_address'] = pcode['Location'] + ", "+ pcode['Post_Office'] + ", " + pcode['Postcode'] + " "+ pcode['State'] + ", " + 'Malaysia'
pcode['locality_address'] 

0        Kangar - Peti surat, Kangar, 01007 Perlis, Mal...
1        Kangar - Beg berkunci, Kangar, 01009 Perlis, M...
2               Alor Redeh, Kangar, 01000 Perlis, Malaysia
3          Alor Tok Terang, Kangar, 01000 Perlis, Malaysia
4              Bohor Gelam, Kangar, 01000 Perlis, Malaysia
                               ...                        
57019        Sungai Bangat, Lawas, 98850 Sarawak, Malaysia
57020      Sungai Berlipat, Lawas, 98850 Sarawak, Malaysia
57021     Sungai Seloating, Lawas, 98850 Sarawak, Malaysia
57022               Trusan, Lawas, 98850 Sarawak, Malaysia
57023               Tuaran, Lawas, 98850 Sarawak, Malaysia
Name: locality_address, Length: 57024, dtype: object

In [112]:
pcode2 = pcode

In [113]:
pcode2.head()

,Location,Post_Office,Postcode,State,locality_address
0,Kangar - Peti surat,Kangar,01007,Perlis,"Kangar - Peti surat, Kangar, 01007 Perlis, Mal..."
1,Kangar - Beg berkunci,Kangar,01009,Perlis,"Kangar - Beg berkunci, Kangar, 01009 Perlis, M..."
2,Alor Redeh,Kangar,01000,Perlis,"Alor Redeh, Kangar, 01000 Perlis, Malaysia"
3,Alor Tok Terang,Kangar,01000,Perlis,"Alor Tok Terang, Kangar, 01000 Perlis, Malaysia"
4,Bohor Gelam,Kangar,01000,Perlis,"Bohor Gelam, Kangar, 01000 Perlis, Malaysia"


In [21]:
# !pip install geopandas -qq
from geopandas.tools import geocode, geocoding, reverse_geocode

### bulk search address coordinate

In [44]:
test = pcode.tail(10)

In [45]:
pcode_lnglat = geocode(strings=test['locality_address'], provider='arcgis')
pcode_lnglat.head()

,geometry,address
57014,POINT (115.45882 4.51105),98850
57015,POINT (115.40517 4.84171),Kampung Temangis
57016,POINT (115.45882 4.51105),98850
57017,POINT (115.58759 4.21642),SK Long Semadoh
57018,POINT (115.48648 4.55248),SK Long Sukang


In [84]:
df = pcode

## Flow mapping address
1. extract postcode

2. have postcode
    - postcode API >> bandar and state <br><br>
    
3. not postcode
    - textcontain dictionary(district) >> district
    - textcontain dictionary(bandar) >> bandar
    - lonlat >> kawasanku API >> district

### Using state/district existing text

In [114]:
pcode2.head()

,Location,Post_Office,Postcode,State,locality_address
0,Kangar - Peti surat,Kangar,01007,Perlis,"Kangar - Peti surat, Kangar, 01007 Perlis, Mal..."
1,Kangar - Beg berkunci,Kangar,01009,Perlis,"Kangar - Beg berkunci, Kangar, 01009 Perlis, M..."
2,Alor Redeh,Kangar,01000,Perlis,"Alor Redeh, Kangar, 01000 Perlis, Malaysia"
3,Alor Tok Terang,Kangar,01000,Perlis,"Alor Tok Terang, Kangar, 01000 Perlis, Malaysia"
4,Bohor Gelam,Kangar,01000,Perlis,"Bohor Gelam, Kangar, 01000 Perlis, Malaysia"


In [115]:
dfn = pcode2

In [116]:
dfn['locality_address'] = dfn['locality_address'].str.upper()

#### extract postcode

In [117]:
# Regular expression to match a continuous number
pattern = r'(\d+)'

# Extract postcode and create a new column
dfn['postcode_new'] = dfn['locality_address'].str.extract(pattern)

In [118]:
dfn.head()

,Location,Post_Office,Postcode,State,locality_address,postcode_new
0,Kangar - Peti surat,Kangar,01007,Perlis,"KANGAR - PETI SURAT, KANGAR, 01007 PERLIS, MAL...",01007
1,Kangar - Beg berkunci,Kangar,01009,Perlis,"KANGAR - BEG BERKUNCI, KANGAR, 01009 PERLIS, M...",01009
2,Alor Redeh,Kangar,01000,Perlis,"ALOR REDEH, KANGAR, 01000 PERLIS, MALAYSIA",01000
3,Alor Tok Terang,Kangar,01000,Perlis,"ALOR TOK TERANG, KANGAR, 01000 PERLIS, MALAYSIA",01000
4,Bohor Gelam,Kangar,01000,Perlis,"BOHOR GELAM, KANGAR, 01000 PERLIS, MALAYSIA",01000


#### mapping with state dictionary

In [119]:
# dfn['negeri_tempat_tinggal'] = [x.split(',')[-1].strip() if pd.notna(x) and ',' in x else '' for x in dfn['alamat_tempat_tinggal']]

statelist = ['JOHOR', 'KEDAH', 'KELANTAN',
'MELAKA', 'NEGERI SEMBILAN',
'PAHANG', 'PERAK', 'PERLIS',
'PULAU PINANG', 'SABAH',
'SARAWAK', 'SELANGOR',
'TERENGGANU',
'PUTRAJAYA',
'LABUAN','KUALA LUMPUR']

dfn['negeri_'] = ''

for state in statelist:
    dfn['locality_address'].fillna('', inplace=True)
    mask = dfn['locality_address'].str.contains(state, case=False)
    dfn.loc[mask, 'negeri_'] = state

In [120]:
dfn.head()

,Location,Post_Office,Postcode,State,locality_address,postcode_new,negeri_
0,Kangar - Peti surat,Kangar,01007,Perlis,"KANGAR - PETI SURAT, KANGAR, 01007 PERLIS, MAL...",01007,PERLIS
1,Kangar - Beg berkunci,Kangar,01009,Perlis,"KANGAR - BEG BERKUNCI, KANGAR, 01009 PERLIS, M...",01009,PERLIS
2,Alor Redeh,Kangar,01000,Perlis,"ALOR REDEH, KANGAR, 01000 PERLIS, MALAYSIA",01000,PERLIS
3,Alor Tok Terang,Kangar,01000,Perlis,"ALOR TOK TERANG, KANGAR, 01000 PERLIS, MALAYSIA",01000,PERLIS
4,Bohor Gelam,Kangar,01000,Perlis,"BOHOR GELAM, KANGAR, 01000 PERLIS, MALAYSIA",01000,PERLIS


#### mapping with district dictionary

In [121]:
# dfn['negeri_tempat_tinggal'] = [x.split(',')[-1].strip() if pd.notna(x) and ',' in x else '' for x in dfn['alamat_tempat_tinggal']]

districtlist = ['KANGAR','ALOR SETAR','LAWAS']

dfn['district_'] = ''

for district in districtlist:
    dfn['locality_address'].fillna('', inplace=True)
    mask = dfn['locality_address'].str.contains(district, case=False)
    dfn.loc[mask, 'district_'] = district

In [122]:
dfn.tail()

,Location,Post_Office,Postcode,State,locality_address,postcode_new,negeri_,district_
57019,Sungai Bangat,Lawas,98850,Sarawak,"SUNGAI BANGAT, LAWAS, 98850 SARAWAK, MALAYSIA",98850,SARAWAK,LAWAS
57020,Sungai Berlipat,Lawas,98850,Sarawak,"SUNGAI BERLIPAT, LAWAS, 98850 SARAWAK, MALAYSIA",98850,SARAWAK,LAWAS
57021,Sungai Seloating,Lawas,98850,Sarawak,"SUNGAI SELOATING, LAWAS, 98850 SARAWAK, MALAYSIA",98850,SARAWAK,LAWAS
57022,Trusan,Lawas,98850,Sarawak,"TRUSAN, LAWAS, 98850 SARAWAK, MALAYSIA",98850,SARAWAK,LAWAS
57023,Tuaran,Lawas,98850,Sarawak,"TUARAN, LAWAS, 98850 SARAWAK, MALAYSIA",98850,SARAWAK,LAWAS


#### mapping with bandar dictionary

In [124]:
bandarlist = ['Ajil',
 'Al Muktatfi Billah Shah',
 'Alor GAjah',
 'Alor Gajah',
 'Alor Setar',
 'Alor gajah',
 'Ampang',
 'Arau',
 'Asahan',
 'Asajaya',
 'Ayer Baloi',
 'Ayer Hitam',
 'Ayer Itam',
 'Ayer Keroh',
 'Ayer Lanas',
 'Ayer Puteh',
 'Ayer Tawar',
 'Ayer Tawar 2',
 'Bachok',
 'Bagan Datoh',
 'Bagan Serai',
 'Bahau',
 'Balik Pulau',
 'Baling',
 'Baling ',
 'Balingian',
 'Balok',
 'Bandar Baharu',
 'Bandar Baru Bangi',
 'Bandar Bera',
 'Bandar Enstek',
 'Bandar Penawar',
 'Bandar Puncak Alam',
 'Bandar Pusat  Jengka',
 'Bandar Pusat Jengka',
 'Bandar Seri Iskandar',
 'Bandar Seri Jempol',
 'Bandar Tenggara',
 'Bandar Tun Abdul Razak',
 'Banting',
 'Baram',
 'Baram ',
 'Batang Kali',
 'Batu Anam',
 'Batu Arang',
 'Batu Caves',
 'Batu Ferringhi',
 'Batu Gajah',
 'Batu Kikir',
 'Batu Kurau',
 'Batu Maung',
 'Batu Pahat',
 'Bau',
 'Bayan Lepas',
 'Beaufort',
 'Bedong',
 'Behrang Stesen',
 'Bekenu',
 'Bekok',
 'Belaga',
 'Belaga ',
 'Belawai',
 'Beluran',
 'Bemban',
 'Benta',
 'Bentong',
 'Benut',
 'Beranang',
 'Bestari Jaya',
 'Betong',
 'Beverly',
 'Bidor',
 'Bintangor',
 'Bintulu',
 'Bongawan',
 'Bota',
 'Brinchang',
 'Bruas',
 'Bukit Besi',
 'Bukit Fraser',
 'Bukit Gambir',
 'Bukit Goh',
 'Bukit Kayu Hitam',
 'Bukit Kuin',
 'Bukit Mertajam',
 'Bukit Pasir',
 'Bukit Payong',
 'Bukit Rotan',
 'Butterworth',
 'Ceneh',
 'Chaah',
 'Chalok',
 'Changkat Jering',
 'Changkat Keruing',
 'Changloon',
 'Chemor',
 'Chenderiang',
 'Chenderong Balai',
 'Chenor',
 'Cherang Ruku',
 'Cheras',
 'Chikus',
 'Chini',
 'Cukai',
 'Cyberjaya',
 'Dabong',
 'Dalat',
 'Damak',
 'Daro',
 'Debak',
 'Dengkil',
 'Dong',
 'Dungun',
 'Durian Tunggal',
 'Endau',
 'Enggor',
 'Engkilili',
 'Gambang',
 'Gelang Patah',
 'Gelugor',
 'Gemas',
 'Gemencheh',
 'Genting Highlands',
 'Gerik',
 'Gerisek',
 'Gopeng',
 'Gua Musang',
 'Gugusan Taib Andak',
 'Gurun',
 'Hulu Langat',
 'Hutan Melintang',
 'Inanam',
 'Intan',
 'Ipoh',
 'Iskandar Puteri',
 'Jasin',
 'Jeli',
 'Jelutong',
 'Jementah',
 'Jeniang',
 'Jenjarom',
 'Jeram',
 'Jerantut',
 'Jerteh',
 'Jitra',
 'Johol',
 'Johor Bahru',
 'Julau',
 'KLIA',
 'Kabong',
 'Kahang',
 'Kajang',
 'Kajang ',
 'Kaki Bukit',
 'Kampar',
 'Kampung Gajah',
 'Kampung Kepayang',
 'Kampung Raja',
 'Kamunting',
 'Kangar',
 'Kanowit',
 'Kapar',
 'Kapit',
 'Karak',
 'Karangan',
 'Kem Desa Pahlawan',
 'Kem Trendak',
 'Kemasek',
 'Kemayan',
 'Keningau',
 'Kepala Batas',
 'Kerling',
 'Kerteh',
 'Ketengah Jaya',
 'Ketereh',
 'Kijal',
 'Klang',
 'Kluang',
 'Kodiang',
 'Kota',
 'Kota Belud',
 'Kota Bharu',
 'Kota Kinabalu',
 'Kota Kinabatangan',
 'Kota Kuala Muda',
 'Kota Marudu',
 'Kota Samarahan',
 'Kota Sarang Semut',
 'Kota Tinggi',
 'Kuala Balah',
 'Kuala Berang',
 'Kuala Besut',
 'Kuala Kangsar',
 'Kuala Kedah',
 'Kuala Ketil',
 'Kuala Klawang',
 'Kuala Krai',
 'Kuala Krau',
 'Kuala Kubu Baru',
 'Kuala Kurau',
 'Kuala Lipis',
 'Kuala Lumpur',
 'Kuala Nerang',
 'Kuala Pegang',
 'Kuala Penyu',
 'Kuala Perlis',
 'Kuala Pilah',
 'Kuala Rompin',
 'Kuala Selangor',
 'Kuala Sepetang',
 'Kuala Sungai Baru',
 'Kuala Terengganu',
 'Kuang',
 'Kuantan',
 'Kuantan ',
 'Kubang Semang',
 'Kuching',
 'Kudat',
 'Kukup',
 'Kulai',
 'Kulim',
 'Kunak',
 'Kupang',
 'Labis',
 'Labu',
 'Labuan',
 'Lahad Datu',
 'Lambor Kanan',
 'Lanchang',
 'Langgar',
 'Langkap',
 'Langkawi',
 'Lawas',
 'Layang-Layang',
 'Lenggong',
 'Likas',
 'Limbang',
 'Lingga',
 'Linggi',
 'Long Lama',
 'Lubok Antu',
 'Lubok China',
 'Lumut',
 'Lunas',
 'Lunas ',
 'Lundu',
 'Lurah Bilut',
 'Lutong',
 'Machang',
 'Malim Nawar',
 'Manong',
 'Mantin',
 'Maran',
 'Marang',
 'Masai',
 'Masjid Tanah',
 'Matang',
 'Matu',
 'Melaka',
 'Melaka ',
 'Melor',
 'Membakut',
 'Mentakab',
 'Menumbok',
 'Merbok',
 'Merlimau',
 'Mersing',
 'Miri',
 'Muadzam Shah',
 'Muar',
 'Mukah',
 'Nabawan',
 'Nanga Medamit',
 'Niah',
 'Niai',
 'Nibong Tebal',
 'Nilai',
 'Nilai ',
 'Padang Besar',
 'Padang Rengas',
 'Padang Serai',
 'Padang Tengku',
 'Pagoh',
 'Paka',
 'Paloh',
 'Pamol',
 'Panchor',
 'Pangkor',
 'Pantai Remis',
 'Papar',
 'Parit',
 'Parit Buntar',
 'Parit Jawa',
 'Parit Raja',
 'Parit Sulong',
 'Pasir Gudang',
 'Pasir Mas',
 'Pasir Puteh',
 'Pekan',
 'Pekan Nenas',
 'Pelabuhan Klang',
 'Penaga',
 'Penampang',
 'Penang Hill',
 'Pendang',
 'Pengerang',
 'Pengkalan Hulu',
 'Perai',
 'Permaisuri',
 'Permatang Pauh',
 'Petaling Jaya',
 'Pokok Sena',
 'Pontian',
 'Port Dickson',
 'Puchong',
 'Pulai Chondong',
 'Pulau Carey',
 'Pulau Indah',
 'Pulau Ketam',
 'Pulau Pinang',
 'Pulau Satu',
 'Pusa',
 'Pusat  Bandar Palong',
 'Pusat Mel Miri',
 'Pusing',
 'Putatan',
 'Putrajaya',
 'Ranau',
 'Ranau ',
 'Rantau',
 'Rantau Panjang',
 'Rasa',
 'Raub',
 'Rawang',
 'Rembau',
 'Rengam',
 'Rengit',
 'Ringlet',
 'Roban',
 'Rompin',
 'Sabak Bernam',
 'Sandakan',
 'Saratok',
 'Sarikei',
 'Sauk',
 'Sebauh',
 'Sebuyau',
 'Sega',
 'Segamat',
 'Sekinchan',
 'Selama',
 'Selandar',
 'Selekoh',
 'Selising',
 'Semenyih',
 'Semerah',
 'Semporna',
 'Senai',
 'Senggarang',
 'Sepang',
 'Serdang',
 'Seremban',
 'Seremban ',
 'Serendah',
 'Seri Gading',
 'Seri Kembangan',
 'Seri Manjong',
 'Seri Manjung',
 'Seri Medan',
 'Serian',
 'Shah Alam',
 'Shah Alam ',
 'Si Rusa',
 'Sibu',
 'Sibu Jaya',
 'Siburan',
 'Sik',
 'Simpang',
 'Simpang Ampat',
 'Simpang Ampat Semanggol',
 'Simpang Durian',
 'Simpang Empat',
 'Simpang Pertang',
 'Simpang Rengam',
 'Simunjan',
 'Sipitang',
 'Sitiawan',
 'Slim River',
 'Song',
 'Spaoh',
 'Sri Aman',
 'Subang Jaya',
 'Sundar',
 'Sungai Ayer Tawar',
 'Sungai Besar',
 'Sungai Buloh',
 'Sungai Jawi',
 'Sungai Koyan',
 'Sungai Lembing',
 'Sungai Mati',
 'Sungai Pelek',
 'Sungai Petani',
 'Sungai Rambai',
 'Sungai Siput',
 'Sungai Sumun',
 'Sungai Tong',
 'Sungai Udang',
 'Sungkai',
 'TLDM Lumut',
 'Taiping',
 'Tambunan',
 'Tamparuli',
 'Tampin',
 'Tanah Merah',
 'Tanah Rata',
 'Tangkak',
 'Tanjong Bungah',
 'Tanjong Ipoh',
 'Tanjong Karang',
 'Tanjong Kling',
 'Tanjong Malim',
 'Tanjong Piandang',
 'Tanjong Rambutan',
 'Tanjong Sepat',
 'Tanjong Tualang',
 'Tanjung Aru',
 'Tanjung Bungah',
 'Tanjung Sepat',
 'Tapah',
 'Tapah Road',
 'Tasek Gelugor',
 'Tasek Gelugur',
 'Tatau',
 'Tawau',
 'Telok Panglima Garang',
 'Teluk Intan',
 'Temangan',
 'Temerloh',
 'Temerloh ',
 'Temoh',
 'Tenghilan',
 'Tenom',
 'Triang',
 'Trolak',
 'Trong',
 'Tronoh',
 'Tuaran',
 'Tumpat',
 'USM Pulau Pinang',
 'Ulu Bernam',
 'Ulu Kinta',
 'Ulu Tiram',
 'Universiti Utara Malaysia',
 'Wakaf Bharu',
 'Yan',
 'Yong Peng']

In [126]:
# Convert all strings to uppercase
bandarlists = [item.upper() for item in bandarlist]

In [127]:
# dfn['negeri_tempat_tinggal'] = [x.split(',')[-1].strip() if pd.notna(x) and ',' in x else '' for x in dfn['alamat_tempat_tinggal']]

# districtlist = ['KANGAR','ALOR SETAR','LAWAS']

dfn['bandar_'] = ''

for bandar in bandarlists:
    dfn['locality_address'].fillna('', inplace=True)
    mask = dfn['locality_address'].str.contains(bandar, case=False)
    dfn.loc[mask, 'bandar_'] = bandar

#### check mapping data

In [131]:
dfn[dfn['negeri_']=='KELANTAN']

,Location,Post_Office,Postcode,State,locality_address,postcode_new,negeri_,district_,bandar_
2695,Kampung Kelantan,Yan,06900,Kedah,"KAMPUNG KELANTAN, YAN, 06900 KEDAH, MALAYSIA",06900,KELANTAN,,YAN
9965,Kota Bharu Trade Center,Kota Bharu,15000,Kelantan,"KOTA BHARU TRADE CENTER, KOTA BHARU, 15000 KEL...",15000,KELANTAN,,KOTA BHARU
9966,Pasar Majlis Pembadaran Kota Bharu,Kota Bharu,15000,Kelantan,"PASAR MAJLIS PEMBADARAN KOTA BHARU, KOTA BHARU...",15000,KELANTAN,,KOTA BHARU
9967,Jalan Stesen Bas,Kota Bharu,15000,Kelantan,"JALAN STESEN BAS, KOTA BHARU, 15000 KELANTAN, ...",15000,KELANTAN,,KOTA BHARU
9968,Jalan Temenggong,Kota Bharu,15000,Kelantan,"JALAN TEMENGGONG, KOTA BHARU, 15000 KELANTAN, ...",15000,KELANTAN,,KOTA BHARU
...,...,...,...,...,...,...,...,...,...
15332,Kampung Pengkalan Merabang,Machang,18500,Kelantan,"KAMPUNG PENGKALAN MERABANG, MACHANG, 18500 KEL...",18500,KELANTAN,,MACHANG
15333,Kampung Pengkalan Petai Dalam,Machang,18500,Kelantan,"KAMPUNG PENGKALAN PETAI DALAM, MACHANG, 18500 ...",18500,KELANTAN,,MACHANG
47891,Kampung Kelantan,Kota Tinggi,81900,Johor,"KAMPUNG KELANTAN, KOTA TINGGI, 81900 JOHOR, MA...",81900,KELANTAN,,KOTA TINGGI
49400,Parit Kelantan,Parit Sulong,83500,Johor,"PARIT KELANTAN, PARIT SULONG, 83500 JOHOR, MAL...",83500,KELANTAN,,PARIT SULONG


wrongly mapping
1. YAN in KEDAH
2. Kampung Kelatan di Kedah dan Kota Tinggi and Muar, Johor
3. Parit Kelantan di Johor

### Using postcode API

In [171]:
import pandas as pd
from tqdm import tqdm
import urllib.request
import json
from shapely.geometry import shape, Point
import time
import warnings

time_start = time.time()
tqdm.pandas()

# Load pcode and dfn DataFrames
pcode = pd.read_csv("https://raw.githubusercontent.com/atqnp/postcode-malaysia/master/postcode/postcode_my.csv")
# Assuming dfn DataFrame is already loaded

# Group the pcode DataFrame by 'Postcode' and convert each group into a dictionary
postcode_to_state_location = pcode.groupby('Postcode').apply(lambda x: {
    'Location': x['Location'].iloc[0],
    'Post_Office': x['Post_Office'].iloc[0],
    'State': x['State'].iloc[0],
}).to_dict()

# Validate the input postcode and lookup the state and location
def get_state_location_info(postcode):
    return postcode_to_state_location.get(postcode, None)

# Progress tracker while processing the DataFrame
total_rows = len(dfn)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")  # Ignore all warnings
    for idx, row in tqdm(dfn.iterrows(), total=total_rows, desc="Processing DataFrame"):
        postcode = row['Postcode']
        state_location_info = get_state_location_info(postcode)

        if state_location_info:
            dfn.at[idx, 'state_new'] = state_location_info['State']
            dfn.at[idx, 'post_office_new'] = state_location_info['Post_Office']
            dfn.at[idx, 'location_new'] = state_location_info['Location']

# Print the updated dfn DataFrame
dfn


Processing DataFrame: 100%|██████████| 57024/57024 [00:12<00:00, 4428.41it/s]


,Location,Post_Office,Postcode,State,locality_address,postcode_new,negeri_,district_,bandar_,state_new,post_office_new,location_new
0,Kangar - Peti surat,Kangar,01007,Perlis,"KANGAR - PETI SURAT, KANGAR, 01007 PERLIS, MAL...",01007,PERLIS,KANGAR,KANGAR,Perlis,Kangar,Kangar - Peti surat
1,Kangar - Beg berkunci,Kangar,01009,Perlis,"KANGAR - BEG BERKUNCI, KANGAR, 01009 PERLIS, M...",01009,PERLIS,KANGAR,KANGAR,Perlis,Kangar,Kangar - Beg berkunci
2,Alor Redeh,Kangar,01000,Perlis,"ALOR REDEH, KANGAR, 01000 PERLIS, MALAYSIA",01000,PERLIS,KANGAR,KANGAR,Perlis,Kangar,Alor Redeh
3,Alor Tok Terang,Kangar,01000,Perlis,"ALOR TOK TERANG, KANGAR, 01000 PERLIS, MALAYSIA",01000,PERLIS,KANGAR,KANGAR,Perlis,Kangar,Alor Redeh
4,Bohor Gelam,Kangar,01000,Perlis,"BOHOR GELAM, KANGAR, 01000 PERLIS, MALAYSIA",01000,PERLIS,KANGAR,KANGAR,Perlis,Kangar,Alor Redeh
...,...,...,...,...,...,...,...,...,...,...,...,...
57019,Sungai Bangat,Lawas,98850,Sarawak,"SUNGAI BANGAT, LAWAS, 98850 SARAWAK, MALAYSIA",98850,SARAWAK,LAWAS,LAWAS,Sarawak,Lawas,Awat-Awat
57020,Sungai Berlipat,Lawas,98850,Sarawak,"SUNGAI BERLIPAT, LAWAS, 98850 SARAWAK, MALAYSIA",98850,SARAWAK,LAWAS,LAWAS,Sarawak,Lawas,Awat-Awat
57021,Sungai Seloating,Lawas,98850,Sarawak,"SUNGAI SELOATING, LAWAS, 98850 SARAWAK, MALAYSIA",98850,SARAWAK,LAWAS,LAWAS,Sarawak,Lawas,Awat-Awat
57022,Trusan,Lawas,98850,Sarawak,"TRUSAN, LAWAS, 98850 SARAWAK, MALAYSIA",98850,SARAWAK,LAWAS,LAWAS,Sarawak,Lawas,Awat-Awat


In [173]:
dfn2 = dfn[['Postcode','State','locality_address','state_new','post_office_new','location_new']]
dfn2

,Postcode,State,locality_address,state_new,post_office_new,location_new
0,01007,Perlis,"KANGAR - PETI SURAT, KANGAR, 01007 PERLIS, MAL...",Perlis,Kangar,Kangar - Peti surat
1,01009,Perlis,"KANGAR - BEG BERKUNCI, KANGAR, 01009 PERLIS, M...",Perlis,Kangar,Kangar - Beg berkunci
2,01000,Perlis,"ALOR REDEH, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh
3,01000,Perlis,"ALOR TOK TERANG, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh
4,01000,Perlis,"BOHOR GELAM, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh
...,...,...,...,...,...,...
57019,98850,Sarawak,"SUNGAI BANGAT, LAWAS, 98850 SARAWAK, MALAYSIA",Sarawak,Lawas,Awat-Awat
57020,98850,Sarawak,"SUNGAI BERLIPAT, LAWAS, 98850 SARAWAK, MALAYSIA",Sarawak,Lawas,Awat-Awat
57021,98850,Sarawak,"SUNGAI SELOATING, LAWAS, 98850 SARAWAK, MALAYSIA",Sarawak,Lawas,Awat-Awat
57022,98850,Sarawak,"TRUSAN, LAWAS, 98850 SARAWAK, MALAYSIA",Sarawak,Lawas,Awat-Awat


In [141]:
dfn2[dfn2['State']=='Kelantan']

,Postcode,State,locality_address,state_new,post_office_new,location_new
9965,15000,Kelantan,"KOTA BHARU TRADE CENTER, KOTA BHARU, 15000 KEL...",Kelantan,Kota Bharu,Kota Bharu Trade Center
9966,15000,Kelantan,"PASAR MAJLIS PEMBADARAN KOTA BHARU, KOTA BHARU...",Kelantan,Kota Bharu,Kota Bharu Trade Center
9967,15000,Kelantan,"JALAN STESEN BAS, KOTA BHARU, 15000 KELANTAN, ...",Kelantan,Kota Bharu,Kota Bharu Trade Center
9968,15000,Kelantan,"JALAN TEMENGGONG, KOTA BHARU, 15000 KELANTAN, ...",Kelantan,Kota Bharu,Kota Bharu Trade Center
9969,15000,Kelantan,"JALAN TENGKU BENDAHARA, KOTA BHARU, 15000 KELA...",Kelantan,Kota Bharu,Kota Bharu Trade Center
...,...,...,...,...,...,...
15329,18500,Kelantan,"KAMPUNG PENGKALAN BERANGAN KAABAH, MACHANG, 18...",Kelantan,Machang,Jalan Kweng Umat
15330,18500,Kelantan,"KAMPUNG PENGKALAN DURIAN DALAM, MACHANG, 18500...",Kelantan,Machang,Jalan Kweng Umat
15331,18500,Kelantan,"KAMPUNG PENGKALAN JETAS, MACHANG, 18500 KELANT...",Kelantan,Machang,Jalan Kweng Umat
15332,18500,Kelantan,"KAMPUNG PENGKALAN MERABANG, MACHANG, 18500 KEL...",Kelantan,Machang,Jalan Kweng Umat


In [143]:
filtered_df = dfn[dfn['locality_address'].str.contains('PARIT KELANTAN, PARIT SULONG, 83500 JOHOR')]
filtered_df

,Location,Post_Office,Postcode,State,locality_address,postcode_new,negeri_,district_,bandar_,state_new,post_office_new,location_new
49400,Parit Kelantan,Parit Sulong,83500,Johor,"PARIT KELANTAN, PARIT SULONG, 83500 JOHOR, MAL...",83500,KELANTAN,,PARIT SULONG,Johor,Parit Sulong,Batu 1 Jalan Yong Peng


### API Kawasanku for District/DUN/Parlimen

In [162]:
dfn2.head()

,Postcode,State,locality_address,state_new,post_office_new,location_new
0,01007,Perlis,"KANGAR - PETI SURAT, KANGAR, 01007 PERLIS, MAL...",Perlis,Kangar,Kangar - Peti surat
1,01009,Perlis,"KANGAR - BEG BERKUNCI, KANGAR, 01009 PERLIS, M...",Perlis,Kangar,Kangar - Beg berkunci
2,01000,Perlis,"ALOR REDEH, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh
3,01000,Perlis,"ALOR TOK TERANG, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh
4,01000,Perlis,"BOHOR GELAM, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh


In [163]:
dfc = dfn2.head(10)
dfc.head()

,Postcode,State,locality_address,state_new,post_office_new,location_new
0,01007,Perlis,"KANGAR - PETI SURAT, KANGAR, 01007 PERLIS, MAL...",Perlis,Kangar,Kangar - Peti surat
1,01009,Perlis,"KANGAR - BEG BERKUNCI, KANGAR, 01009 PERLIS, M...",Perlis,Kangar,Kangar - Beg berkunci
2,01000,Perlis,"ALOR REDEH, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh
3,01000,Perlis,"ALOR TOK TERANG, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh
4,01000,Perlis,"BOHOR GELAM, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh


In [174]:
# !pip install geopandas -qq
from geopandas.tools import geocode, geocoding, reverse_geocode
import pandas as pd
from shapely.geometry import Point

dfc[['coordinate','point_address']] = geocode(strings=dfc['locality_address'], provider='arcgis')

# # Extract lon and lat
dfc['lon'] = dfc['coordinate'].apply(lambda point: point.x)
dfc['lat'] = dfc['coordinate'].apply(lambda point: point.y)

# # # Drop the original geometry column
dfc.drop(columns=['coordinate'], inplace=True)

dfc.head()


C:\Users\najmi.ariffin\AppData\Local\Temp\4\ipykernel_50720\13317225.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc[['coordinate','point_address']] = geocode(strings=dfc['locality_address'], provider='arcgis')
C:\Users\najmi.ariffin\AppData\Local\Temp\4\ipykernel_50720\13317225.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc['lon'] = dfc['coordinate'].apply(lambda point: point.x)
C:\Users\najmi.ariffin\AppData\Local\Temp\4\ipykernel_50720\13317225.py:10: SettingWithCopyWarning: 
A value is 

,Postcode,State,locality_address,state_new,post_office_new,location_new,point_address,lon,lat,state,district,parlimen,dun
0,01007,Perlis,"KANGAR - PETI SURAT, KANGAR, 01007 PERLIS, MAL...",Perlis,Kangar,Kangar - Peti surat,"Kangar, Perlis",100.194270,6.440040,Perlis,Perlis,P.002 Kangar,N.08 Indera Kayangan
1,01009,Perlis,"KANGAR - BEG BERKUNCI, KANGAR, 01009 PERLIS, M...",Perlis,Kangar,Kangar - Beg berkunci,"Jalan Kangar, Taman Temak Jaya, Seriab, Kangar...",100.184701,6.423744,Perlis,Perlis,P.002 Kangar,N.10 Kayang
2,01000,Perlis,"ALOR REDEH, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh,01000,100.196655,6.436264,Perlis,Perlis,P.002 Kangar,N.08 Indera Kayangan
3,01000,Perlis,"ALOR TOK TERANG, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh,01000,100.196655,6.436264,Perlis,Perlis,P.002 Kangar,N.08 Indera Kayangan
4,01000,Perlis,"BOHOR GELAM, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh,01000,100.196655,6.436264,Perlis,Perlis,P.002 Kangar,N.08 Indera Kayangan


In [175]:
import pandas as pd
from tqdm import tqdm
import urllib.request
import json
from shapely.geometry import shape, Point
import time

time_start = time.time()
tqdm.pandas()

PATH_GEOJSON = 'https://raw.githubusercontent.com/dosm-malaysia/data-open/main/datasets/geodata/'

geojsons = ['administrative_0_malaysia',
            'administrative_1_state',
            'administrative_2_district',
            'electoral_0_parlimen',
            'electoral_1_dun']

for i in range (len(geojsons)): geojsons[i] = PATH_GEOJSON + geojsons[i] + '.geojson'

states = json.load(urllib.request.urlopen(geojsons[1]))
districts = json.load(urllib.request.urlopen(geojsons[2]))
parlimens = json.load(urllib.request.urlopen(geojsons[3]))
duns = json.load(urllib.request.urlopen(geojsons[4]))
int_jsonfile = {1: states, 2: districts, 3: parlimens, 4: duns}

def reverse_geocode(lon,lat,geojson_file,name_field):
    try: point = Point(lon, lat)
    except Exception as e:
        print(e)
        return 'Error'

    for feature in geojson_file['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point): return (feature['properties'])[name_field].title()

    return 'OUT_OF_BOUNDS'

In [176]:
import warnings

# Ignore RuntimeWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


In [177]:
levels = ['country','state','district','parlimen','dun']
for i in [1,2,3,4]: 
    dfc[levels[i]] = dfc.progress_apply(lambda x: reverse_geocode(x['lon'], x['lat'], int_jsonfile[i], levels[i]), axis = 1)

dfc.head()

100%|██████████| 10/10 [00:00<00:00, 21.98it/s]
C:\Users\najmi.ariffin\AppData\Local\Temp\4\ipykernel_50720\4017863061.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc[levels[i]] = dfc.progress_apply(lambda x: reverse_geocode(x['lon'], x['lat'], int_jsonfile[i], levels[i]), axis = 1)
100%|██████████| 10/10 [00:00<00:00, 19.42it/s]
C:\Users\najmi.ariffin\AppData\Local\Temp\4\ipykernel_50720\4017863061.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc[levels[i]] = dfc.progress_apply(lambda x: rev

,Postcode,State,locality_address,state_new,post_office_new,location_new,point_address,lon,lat,state,district,parlimen,dun
0,01007,Perlis,"KANGAR - PETI SURAT, KANGAR, 01007 PERLIS, MAL...",Perlis,Kangar,Kangar - Peti surat,"Kangar, Perlis",100.194270,6.440040,Perlis,Perlis,P.002 Kangar,N.08 Indera Kayangan
1,01009,Perlis,"KANGAR - BEG BERKUNCI, KANGAR, 01009 PERLIS, M...",Perlis,Kangar,Kangar - Beg berkunci,"Jalan Kangar, Taman Temak Jaya, Seriab, Kangar...",100.184701,6.423744,Perlis,Perlis,P.002 Kangar,N.10 Kayang
2,01000,Perlis,"ALOR REDEH, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh,01000,100.196655,6.436264,Perlis,Perlis,P.002 Kangar,N.08 Indera Kayangan
3,01000,Perlis,"ALOR TOK TERANG, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh,01000,100.196655,6.436264,Perlis,Perlis,P.002 Kangar,N.08 Indera Kayangan
4,01000,Perlis,"BOHOR GELAM, KANGAR, 01000 PERLIS, MALAYSIA",Perlis,Kangar,Alor Redeh,01000,100.196655,6.436264,Perlis,Perlis,P.002 Kangar,N.08 Indera Kayangan
